# RENABAP

## Objectivos

## Fuentes de datos

### Datos

#### RENABAP

#### Censo Argentino

#### GHSL

#### Google-Microsoft Open Buildings

### Metodología

#### Interpolación por area

#### Mapeo dasymetrico

## Metodología y procesamiento


In [ ]:
import pandas as pd
import geopandas as gpd
import requests
from io import StringIO

import boto3
import duckdb


import matplotlib.pyplot as plt
import contextily as ctx

import numpy as np
import s2sphere
from botocore.config import Config
from rasterstats import zonal_stats

from io import BytesIO
from owslib.wfs import WebFeatureService
from shapely.geometry import box


USE_CRS = "EPSG:5349"

# Generic mapping functions
def setup_base_map(figsize=(12, 10), bounds=None, padding_x=500, padding_y=500):
    """Create figure and set up basic map boundaries with padding."""
    if bounds is None:
        bounds = renabap_pba_intersect.total_bounds
    
    fig, ax = plt.subplots(figsize=figsize)
    ax.set_xlim(bounds[0] - padding_x, bounds[2] + padding_x)
    ax.set_ylim(bounds[1] - padding_y, bounds[3] + padding_y)
    return fig, ax

def add_basemap(ax, zoom=13):
    """Add CartoDB basemap to the axes."""
    # Convert to Web Mercator for basemap
    ax_web_mercator = ax.figure.add_axes(ax.get_position(), projection='EPSG:3857')
    ax_web_mercator.set_xlim(ax.get_xlim())
    ax_web_mercator.set_ylim(ax.get_ylim())
    
    ctx.add_basemap(
        ax_web_mercator,
        source=ctx.providers.CartoDB.PositronNoLabels,
        zorder=0,
        zoom=zoom,
    )
    
    # Copy basemap to original axes
    ax.imshow(ax_web_mercator.get_images()[0], extent=ax.get_xlim() + ax.get_ylim(), zorder=0)
    ax.figure.delaxes(ax_web_mercator)
    
    return ax

def add_north_arrow(ax, x=0.95, y=0.95, width=0.05, height=0.05):
    """Add a north arrow to the map."""
    # Convert to axes coordinates
    trans = ax.transAxes
    
    # North arrow
    arrow_x = [x, x, x + width/2, x, x - width/2]
    arrow_y = [y - height, y, y + height, y, y + height]
    
    ax.fill(arrow_x, arrow_y, transform=trans, color='black', zorder=1000)
    ax.text(x, y + height + 0.02, 'N', transform=trans, 
            ha='center', va='bottom', fontweight='bold', fontsize=12, zorder=1000)

def create_consistent_map(title, bounds=None):
    """Create a map with consistent styling and basemap."""
    fig, ax = setup_base_map(bounds=bounds)
    
    # Add basemap
    add_basemap(ax)
    
    # Add north arrow
    add_north_arrow(ax)
    
    # Set title
    ax.set_title(title, fontsize=16, fontweight='bold', pad=20)
    
    # Remove axes
    ax.set_axis_off()
    
    return fig, ax

### import data

response = requests.get(
    "https://www.argentina.gob.ar/sites/default/files/renabap-2023-12-06.geojson"
)
renabap = gpd.read_file(StringIO(response.text))
renabap_pba = renabap[renabap["provincia"] == "Buenos Aires"]
renabap_pba = renabap_pba.to_crs(USE_CRS)

peligro_path = "/home/nissim/Documents/dev/fulbright/ciut-riesgo/notebooks/data/la_plata_pelig_2023_datos_originales.geojson"
peligro = gpd.read_file(peligro_path)
peligro = peligro.to_crs(USE_CRS)

peligro_bounds = peligro.total_bounds
peligro_bbox = box(*peligro_bounds)

renabap_pba_intersect = renabap_pba[
    renabap_pba.geometry.intersects(peligro_bbox)
].copy()



def wfs_to_gdf(wfs_url: str, layer_name: str, srs: str = "EPSG:4326") -> gpd.GeoDataFrame:
    """
    Descarga una capa WFS y la devuelve como GeoDataFrame.
    
    Args:
        wfs_url (str): URL del servicio WFS.
        layer_name (str): Nombre de la capa (typename).
        srs (str): Código EPSG del sistema de referencia de coordenadas.
    
    Returns:
        gpd.GeoDataFrame: Capa descargada como GeoDataFrame.
    """
    wfs = WebFeatureService(url=wfs_url, version="2.0.0")
    response = wfs.getfeature(typename=layer_name, srsname=srs)
    gdf = gpd.read_file(BytesIO(response.read()))
    return gdf

partidos = wfs_to_gdf(
     wfs_url="https://geo.arba.gov.ar/geoserver/idera/wfs",
     layer_name="idera:Departamento",
     srs="EPSG:5347"
)

partidos = partidos.to_crs(USE_CRS)

la_plata = partidos[partidos['fna'] == 'Partido de La Plata']

# Get common bounds for all maps
common_bounds = renabap_pba_intersect.total_bounds

### Mapa 1: Asentamientos RENABAP en La Plata

```{python}
# Map 1: RENABAP settlements overlaid on La Plata partido
fig1, ax1 = create_consistent_map("Asentamientos RENABAP en La Plata", common_bounds)
plt.tight_layout()
plt.show()

### Mapa 2: Datos de población GHSL


In [ ]:
# Map 2: GHSL population data
fig2, ax2 = create_consistent_map("Datos de población GHSL", common_bounds)
plt.tight_layout()
plt.show()

In [ ]:
# Map 1: RENABAP settlements overlaid on La Plata partido
fig1, ax1 = create_consistent_map("Asentamientos RENABAP en La Plata", common_bounds)

# Add La Plata partido boundary
la_plata.boundary.plot(ax=ax1, color='darkblue', linewidth=2, zorder=5, label='Partido de La Plata')

# Add RENABAP settlements as outlines
renabap_pba_intersect.boundary.plot(ax=ax1, color='red', linewidth=1, zorder=6, label='Asentamientos RENABAP')

ax1.legend(loc='upper right')
plt.tight_layout()
plt.show()

### Interpolación por area


In [ ]:
# Ensure both GeoDataFrames have the same CRS
if renabap_pba_intersect.crs != peligro.crs:
    peligro = peligro.to_crs(renabap_pba_intersect.crs)

# Get unique hazard levels
hazard_levels = peligro["PELIGROSID"].unique()

# Initialize result columns
renabap_with_porciones = renabap_pba_intersect.copy()
for level in hazard_levels:
    renabap_with_porciones[f"porcion_{level}"] = 0.0

# Calculate total area of each barrio
renabap_with_porciones['total_area'] = renabap_with_porciones.geometry.area

# For each barrio, calculate intersection with each hazard level
for idx, barrio in renabap_with_porciones.iterrows():
    barrio_geom = barrio.geometry
    barrio_total_area = barrio_geom.area
    
    if barrio_total_area == 0:
        continue
        
    for level in hazard_levels:
        hazard_subset = peligro[peligro["PELIGROSID"] == level]
        
        if hazard_subset.empty:
            continue
        
        # Calculate intersection area
        intersection_area = 0
        for _, hazard_row in hazard_subset.iterrows():
            try:
                intersection = barrio_geom.intersection(hazard_row.geometry)
                if not intersection.is_empty:
                    intersection_area += intersection.area
            except Exception as e:
                print(f"Error calculating intersection for {barrio.get('nombre_barrio', idx)}: {e}")
                continue
        
        # Calculate proportion
        proportion = intersection_area / barrio_total_area if barrio_total_area > 0 else 0
        renabap_with_porciones.at[idx, f"porcion_{level}"] = proportion

# Create tidy format dataframe
renabap_tidy = []

for idx, row in renabap_with_porciones.iterrows():
    for level in hazard_levels:
        familias_expuestas = row[f"porcion_{level}"] * row["familias_aproximadas"]
        
        renabap_tidy.append({
            "id_renabap": row["id_renabap"],
            "nombre_barrio": row["nombre_barrio"],
            "peligrosidad": level,
            "fam_expuestas_areal": familias_expuestas
        })

renabap_tidy = pd.DataFrame(renabap_tidy)

### Cuenta de edificios


In [ ]:
def fetch_buildings(geodataframe, temp_file="buildings_filtered.parquet"):
    """Fetch building data for a given GeoDataFrame region"""

    # Get S2 cell and bounds
    center = geodataframe.to_crs("epsg:3857").union_all().centroid
    center_wgs84 = (
        gpd.GeoDataFrame(geometry=[center], crs="EPSG:3857")
        .to_crs(epsg=4326)
        .geometry.iloc[0]
    )
    cell = s2sphere.CellId.from_lat_lng(
        s2sphere.LatLng.from_degrees(center_wgs84.y, center_wgs84.x)
    ).parent(10)
    bounds = geodataframe.to_crs("epsg:4326").total_bounds

    # Find matching S2 partition
    s3 = boto3.client(
        "s3",
        endpoint_url="https://data.source.coop",
        aws_access_key_id="",
        aws_secret_access_key="",
        config=Config(s3={"addressing_style": "path"}),
    )

    partitions = {
        obj["Key"].split("/")[-1].replace(".parquet", "")
        for obj in s3.list_objects_v2(
            Bucket="vida",
            Prefix="google-microsoft-open-buildings/geoparquet/by_country_s2/country_iso=ARG/",
        ).get("Contents", [])
    }

    parent_id = next(
        str(cell.parent(level).id())
        for level in range(10, 0, -1)
        if str(cell.parent(level).id()) in partitions
    )

    # Setup DuckDB and query
    con = duckdb.connect()
    for cmd in [
        "INSTALL spatial",
        "LOAD spatial",
        "INSTALL httpfs",
        "LOAD httpfs",
        "SET s3_region='us-east-1'",
        "SET s3_endpoint='data.source.coop'",
        "SET s3_use_ssl=true",
        "SET s3_url_style='path'",
    ]:
        con.execute(cmd)

    # Export and read back
    query = f"""
    COPY (SELECT * FROM 's3://vida/google-microsoft-open-buildings/geoparquet/by_country_s2/country_iso=ARG/{parent_id}.parquet'
          WHERE bbox.xmax >= {bounds[0]} AND bbox.xmin <= {bounds[2]} AND
                bbox.ymax >= {bounds[1]} AND bbox.ymin <= {bounds[3]}
    ) TO '{temp_file}' (FORMAT PARQUET);
    """

    con.execute(query)
    df = pd.read_parquet(temp_file)
    df["geometry"] = gpd.GeoSeries.from_wkb(df["geometry"])

    return gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")


# Usage:
buildings = fetch_buildings(renabap_pba_intersect)

### Mapeo dasymetrico con datos GHSL


In [ ]:
import rioxarray
from shapely.geometry import box

# Load GHSL data with dask chunking for memory efficiency
ghsl = rioxarray.open_rasterio(
    "/home/nissim/Downloads/spatial/GHS_POP_E2025_GLOBE_R2023A_54009_100_V1_0_R14_C13/GHS_POP_E2025_GLOBE_R2023A_54009_100_V1_0_R14_C13.tif",
    chunks={"x": 1024, "y": 1024},  # Adjust chunk size based on your memory constraints
)

# Reproject to your target CRS with streaming
ghsl = ghsl.rio.reproject(dst_crs=USE_CRS)

# Clip to renabap_pba_intersect bounding box using streaming
bounding_box = box(
    *renabap_pba_intersect.total_bounds
)  # Create a box from the bounding box coordinates

ghsl_clipped = ghsl.rio.clip(
    [bounding_box],  # Use the bounding box as a geometry (wrapped in a list)
    from_disk=True,  # Process from disk to avoid loading entire dataset into memory
)


import rasterstats

# Step 1: Calculate the total GHSL population per barrio popular using zonal statistics

# Convert to the format expected by rasterstats
geometries = [geom for geom in renabap_pba_intersect.geometry]

# Use rasterstats for vectorized zonal statistics
stats = rasterstats.zonal_stats(
    geometries,
    ghsl_clipped.values[0],  # rasterstats expects 2D array
    affine=ghsl_clipped.rio.transform(),
    stats=["sum"],
    nodata=ghsl_clipped.rio.nodata,
)

# Extract the sum values
ghsl_totals = [stat["sum"] if stat["sum"] is not None else 0 for stat in stats]

# Add the GHSL population estimates as a new column
renabap_pba_intersect["ghsl_pop_est"] = ghsl_totals

from rasterio.features import rasterize
import numpy as np

# Get the reference raster properties from GHSL data
reference_raster = ghsl_clipped
reference_transform = reference_raster.rio.transform()
reference_crs = reference_raster.rio.crs
reference_shape = reference_raster.shape[1:]  # Get 2D shape (height, width)


# Prepare geometries and values for rasterization
geometries_ghsl = [
    (geom, value)
    for geom, value in zip(
        renabap_pba_intersect.geometry, renabap_pba_intersect["ghsl_pop_est"]
    )
]
geometries_familias = [
    (geom, value)
    for geom, value in zip(
        renabap_pba_intersect.geometry, renabap_pba_intersect["familias_aproximadas"]
    )
]

# Create GHSL population raster
ghsl_pop_raster = rasterize(
    geometries_ghsl,
    out_shape=reference_shape,
    transform=reference_transform,
    fill=0,
    dtype=np.float32,
    all_touched=False,
)

# Create familias aproximadas raster
familias_raster = rasterize(
    geometries_familias,
    out_shape=reference_shape,
    transform=reference_transform,
    fill=0,
    dtype=np.float32,
    all_touched=False,
)


# Step 1: Divide original GHSL by the barrio-level GHSL to get fractional population
# Use masking to avoid division on invalid cells
mask = (ghsl_clipped.values[0] != -200) & (ghsl_pop_raster > 0.1)
ghsl_fractional = np.full_like(ghsl_clipped.values[0], -200, dtype=np.float64)
ghsl_fractional[mask] = ghsl_clipped.values[0][mask] / ghsl_pop_raster[mask]

# Step 2: Multiply fractional population by familias aproximadas to get downscaled data
mask2 = (ghsl_fractional != -200) & (familias_raster > 0)
familias_downscaled = np.full_like(ghsl_clipped.values[0], -200, dtype=np.float64)
familias_downscaled[mask2] = ghsl_fractional[mask2] * familias_raster[mask2]

# Verify the results - exclude -200 from range calculations
ghsl_valid = ghsl_clipped.values[0] != -200
fractional_valid = ghsl_fractional != -200
downscaled_valid = familias_downscaled != -200



# Check that the sum of downscaled familias equals the original familias aproximadas
total_original_familias = renabap_pba_intersect["familias_aproximadas"].sum()
total_downscaled_familias = np.sum(familias_downscaled[downscaled_valid])

# Intersect settlements with hazard zones
settlement_hazard = gpd.overlay(renabap_pba_intersect, peligro, how="intersection")

# Create GHSL tidy dataframe with matching structure
ghsl_tidy = []

for idx, row in settlement_hazard.iterrows():
    stats = zonal_stats(
        [row.geometry],
        familias_downscaled,  # your numpy array
        affine=reference_transform,  # get transform from your xarray
        stats=["sum"],
        nodata=-200,  # use your actual nodata value
    )[0]

    ghsl_tidy.append(
        {
            "id_renabap": row["id_renabap"],
            "peligrosidad": row["PELIGROSID"],
            "fam_expuestas_ghsl": stats["sum"] if stats["sum"] is not None else 0,
        }
    )

ghsl_tidy = pd.DataFrame(ghsl_tidy)

### Mapa 3: Huellas de edificios

```{python}
# Map 3: Building footprints
fig3, ax3 = create_consistent_map("Huellas de edificios", common_bounds)
plt.tight_layout()
plt.show()

In [ ]:
# Map 2: GHSL population data
fig2, ax2 = create_consistent_map("Datos de población GHSL", common_bounds)

# Plot GHSL data as raster
ghsl_clipped.plot.imshow(ax=ax2, cmap='YlOrRd', zorder=5, add_colorbar=True, 
                         cbar_kwargs={'label': 'Población estimada'})

plt.tight_layout()
plt.show()

### Mapeo dasymetrico con datos de edificios


In [ ]:
# Reproject buildings to match the analysis CRS
buildings_proj = buildings.to_crs(USE_CRS)

# Step 1: Calculate buildings per settlement-hazard intersection
buildings_hazard = gpd.overlay(buildings_proj, settlement_hazard, how="intersection")

# Count buildings per settlement-hazard combination
buildings_per_hazard = (
    buildings_hazard.groupby(["id_renabap", "PELIGROSID"])
    .size()
    .reset_index(name="buildings_count")
)

# Step 2: Calculate total buildings per settlement (barrio popular)
buildings_settlement = gpd.overlay(
    buildings_proj, renabap_pba_intersect, how="intersection"
)
total_buildings_per_settlement = (
    buildings_settlement.groupby("id_renabap")
    .size()
    .reset_index(name="total_buildings")
)

# Step 3: Merge and calculate ratios
hazard_ratios = buildings_per_hazard.merge(
    total_buildings_per_settlement, on="id_renabap", how="left"
)
hazard_ratios["building_ratio"] = (
    hazard_ratios["buildings_count"] / hazard_ratios["total_buildings"]
)

# Step 4: Get total population per settlement and apply ratios
settlement_population = renabap_pba_intersect[
    ["id_renabap", "familias_aproximadas"]
].copy()

# Merge with ratios and calculate population estimates
population_estimates = hazard_ratios.merge(
    settlement_population, on="id_renabap", how="left"
)
population_estimates["estimated_population_hazard"] = (
    population_estimates["building_ratio"]
    * population_estimates["familias_aproximadas"]
)

# Step 5: Create final results with totals
final_results = population_estimates[
    ["id_renabap", "PELIGROSID", "estimated_population_hazard"]
].copy()

# Add total population rows (no hazard breakdown)
total_pop_rows = settlement_population.copy()
total_pop_rows["PELIGROSID"] = "total"
total_pop_rows["estimated_population_hazard"] = total_pop_rows["familias_aproximadas"]

# Combine
final_results = pd.concat(
    [
        final_results,
        total_pop_rows[["id_renabap", "PELIGROSID", "estimated_population_hazard"]],
    ],
    ignore_index=True,
)

# Create buildings tidy dataframe with matching structure
buildings_tidy = final_results[
    ["id_renabap", "PELIGROSID", "estimated_population_hazard"]
].copy()

# Rename columns to match the structure
buildings_tidy = buildings_tidy.rename(
    columns={
        "PELIGROSID": "peligrosidad",
        "estimated_population_hazard": "fam_expuestas_edificios",
    }
)

# Filter out the 'total' rows since we only want hazard-specific data
buildings_tidy = buildings_tidy[buildings_tidy["peligrosidad"] != "total"].copy()

### Mapa 3: Huellas de edificios

```{python}
# Map 3: Building footprints
fig3, ax3 = create_consistent_map("Huellas de edificios", common_bounds)

# Plot building footprints
buildings_proj.plot(ax=ax3, color='darkgreen', alpha=0.6, markersize=1, zorder=5)

plt.tight_layout()
plt.show()

In [ ]:
def setup_base_map(figsize=(12, 10), bounds=None, padding_x=500, padding_y=500):
    """Create figure and set up basic map boundaries with padding."""
    if bounds is None:
        bounds = renabap_pba_intersect.total_bounds
    
    fig, ax = plt.subplots(figsize=figsize)
    ax.set_xlim(bounds[0] - padding_x, bounds[2] + padding_x)
    ax.set_ylim(bounds[1] - padding_y, bounds[3] + padding_y)
    return fig, ax

def add_basemap(ax, zoom=13):
    """Add CartoDB basemap to the axes."""
    # Convert to Web Mercator for basemap
    ax_web_mercator = ax.figure.add_axes(ax.get_position(), projection='EPSG:3857')
    ax_web_mercator.set_xlim(ax.get_xlim())
    ax_web_mercator.set_ylim(ax.get_ylim())
    
    ctx.add_basemap(
        ax_web_mercator,
        source=ctx.providers.CartoDB.PositronNoLabels,
        zorder=0,
        zoom=zoom,
    )
    
    # Copy basemap to original axes
    ax.imshow(ax_web_mercator.get_images()[0], extent=ax.get_xlim() + ax.get_ylim(), zorder=0)
    ax.figure.delaxes(ax_web_mercator)
    
    return ax

def add_north_arrow(ax, x=0.95, y=0.95, width=0.05, height=0.05):
    """Add a north arrow to the map."""
    # Convert to axes coordinates
    trans = ax.transAxes
    
    # North arrow
    arrow_x = [x, x, x + width/2, x, x - width/2]
    arrow_y = [y - height, y, y + height, y, y + height]
    
    ax.fill(arrow_x, arrow_y, transform=trans, color='black', zorder=1000)
    ax.text(x, y + height + 0.02, 'N', transform=trans, 
            ha='center', va='bottom', fontweight='bold', fontsize=12, zorder=1000)

def create_consistent_map(title, bounds=None):
    """Create a map with consistent styling and basemap."""
    fig, ax = setup_base_map(bounds=bounds)
    
    # Add basemap
    add_basemap(ax)
    
    # Add north arrow
    add_north_arrow(ax)
    
    # Set title
    ax.set_title(title, fontsize=16, fontweight='bold', pad=20)
    
    # Remove axes
    ax.set_axis_off()
    
    return fig, ax

# Get common bounds for all maps
common_bounds = renabap_pba_intersect.total_bounds

# Map 1: RENABAP settlements overlaid on La Plata partido
fig1, ax1 = create_consistent_map("Asentamientos RENABAP en La Plata", common_bounds)

# Add La Plata partido boundary
la_plata.boundary.plot(ax=ax1, color='darkblue', linewidth=2, zorder=5, label='Partido de La Plata')

# Add RENABAP settlements as outlines
renabap_pba_intersect.boundary.plot(ax=ax1, color='red', linewidth=1, zorder=6, label='Asentamientos RENABAP')

ax1.legend(loc='upper right')
plt.tight_layout()
plt.show()

# Map 2: GHSL population data
fig2, ax2 = create_consistent_map("Datos de población GHSL", common_bounds)

# Plot GHSL data as raster
ghsl_clipped.plot.imshow(ax=ax2, cmap='YlOrRd', zorder=5, add_colorbar=True, 
                         cbar_kwargs={'label': 'Población estimada'})

plt.tight_layout()
plt.show()

# Map 3: Building footprints
fig3, ax3 = create_consistent_map("Huellas de edificios", common_bounds)

# Plot building footprints
buildings_proj.plot(ax=ax3, color='darkgreen', alpha=0.6, markersize=1, zorder=5)

plt.tight_layout()
plt.show()

# Map 4: Barrios por población expuesta estimada
fig4, ax4 = create_consistent_map("Barrios por población expuesta estimada", common_bounds)

# Calculate total exposure per barrio (using area method as example)
barrio_exposure = (
    final_tidy[final_tidy["metodo"] == "area"]
    .groupby("id_renabap")["fam_expuestas"]
    .sum()
    .reset_index()
)

# Merge with geometry
barrio_exposure_map = renabap_pba_intersect.merge(
    barrio_exposure, on="id_renabap", how="left"
)

# Plot choropleth
barrio_exposure_map.plot(
    column="fam_expuestas", 
    ax=ax4, 
    legend=True, 
    legend_kwds={'label': 'Familias expuestas'},
    cmap='Reds',
    zorder=5
)

plt.tight_layout()
plt.show()

### Mapa 4: Barrios por población expuesta estimada


In [ ]:
# Map 4: Barrios por población expuesta estimada
fig4, ax4 = create_consistent_map("Barrios por población expuesta estimada", common_bounds)
plt.tight_layout()
plt.show()

## Resultados y conclusiones


In [ ]:
# Join all three dataframes by id_renabap and peligrosidad
final_df = renabap_tidy.merge(
    ghsl_tidy, on=["id_renabap", "peligrosidad"], how="outer"
)
final_df = final_df.merge(
    buildings_tidy, on=["id_renabap", "peligrosidad"], how="outer"
)

# Impute 0s for NA values in fam_expuestas columns
fam_expuestas_columns = [col for col in final_df.columns if 'fam_expuestas' in col]
final_df[fam_expuestas_columns] = final_df[fam_expuestas_columns].fillna(0)

# Create long format dataframe with aggregation
final_tidy = []

# Add renabap data
for _, row in renabap_tidy.iterrows():
    final_tidy.append(
        {
            "id_renabap": row["id_renabap"],
            "peligrosidad": row["peligrosidad"],
            "metodo": "area",
            "fam_expuestas": row["fam_expuestas_areal"],
        }
    )

# Add ghsl data
for _, row in ghsl_tidy.iterrows():
    final_tidy.append(
        {
            "id_renabap": row["id_renabap"],
            "peligrosidad": row["peligrosidad"],
            "metodo": "ghsl",
            "fam_expuestas": row["fam_expuestas_ghsl"],
        }
    )

# Add buildings data
for _, row in buildings_tidy.iterrows():
    final_tidy.append(
        {
            "id_renabap": row["id_renabap"],
            "peligrosidad": row["peligrosidad"],
            "metodo": "edificios",
            "fam_expuestas": row["fam_expuestas_edificios"],
        }
    )

final_tidy = pd.DataFrame(final_tidy)

# Aggregate to get one observation per barrio per hazard level per method
final_tidy = (
    final_tidy.groupby(["id_renabap", "peligrosidad", "metodo"])["fam_expuestas"]
    .sum()
    .reset_index()
)

# Create complete combination of all barrios, hazard levels, and methods
all_barrios = final_tidy["id_renabap"].unique()
all_hazard_levels = ["alta", "baja", "media"]
all_methods = ["area", "ghsl", "edificios"]

complete_combinations = pd.DataFrame([
    {"id_renabap": barrio, "peligrosidad": hazard, "metodo": method}
    for barrio in all_barrios
    for hazard in all_hazard_levels
    for method in all_methods
])

# Merge with actual data and fill missing values with 0
final_tidy = complete_combinations.merge(
    final_tidy, on=["id_renabap", "peligrosidad", "metodo"], how="left"
)
final_tidy["fam_expuestas"] = final_tidy["fam_expuestas"].fillna(0)

# Calculate total exposure per hazard level per method
summary = (
    final_tidy.groupby(["peligrosidad", "metodo"])["fam_expuestas"]
    .sum()
    .reset_index()
    .pivot(index="peligrosidad", columns="metodo", values="fam_expuestas")
)

import matplotlib.pyplot as plt
import seaborn as sns

# Filter for high exposure (alta peligrosidad)
alta_data = final_tidy[final_tidy["peligrosidad"] == "alta"].copy()

# Calculate total exposure per barrio across all methods
total_exposure = (
    alta_data.groupby("id_renabap")["fam_expuestas"]
    .sum()
    .sort_values(ascending=False)
)
top_25_barrios = total_exposure.head(25).index

# Filter data for top 25 barrios
top_25_data = alta_data[
    alta_data["id_renabap"].isin(top_25_barrios)
].copy()

# Create range plot showing min, max, and individual points
plt.figure(figsize=(15, 10))

# Define colors for methods
method_colors = {"area": "blue", "ghsl": "red", "edificios": "green"}

for i, barrio in enumerate(top_25_barrios):
    barrio_data = top_25_data[top_25_data["id_renabap"] == barrio]
    if len(barrio_data) > 0:
        values = barrio_data["fam_expuestas"].values
        min_val = values.min()
        max_val = values.max()

        # Plot range line
        plt.plot([min_val, max_val], [i, i], "k-", alpha=0.5, linewidth=2)

        # Plot individual points colored by method
        for _, row in barrio_data.iterrows():
            color = method_colors[row["metodo"]]
            plt.plot(row["fam_expuestas"], i, "o", color=color, markersize=6, alpha=0.8)

plt.yticks(range(len(top_25_barrios)), top_25_barrios)
plt.xlabel("Familias Expuestas")
plt.ylabel("Barrio ID")
plt.title("Range of High Exposure Estimates for Top 25 Barrios", fontsize=14)
plt.grid(True, alpha=0.3)

# Add legend
legend_elements = [
    plt.Line2D(
        [0],
        [0],
        marker="o",
        color="w",
        markerfacecolor=color,
        markersize=8,
        label=method,
    )
    for method, color in method_colors.items()
]
plt.legend(handles=legend_elements, title="Método")

plt.tight_layout()
plt.show()

### Mapa 4: Barrios por población expuesta estimada


In [ ]:
# Map 4: Barrios por población expuesta estimada
fig4, ax4 = create_consistent_map("Barrios por población expuesta estimada", common_bounds)

# Calculate total exposure per barrio (using area method as example)
barrio_exposure = (
    final_tidy[final_tidy["metodo"] == "area"]
    .groupby("id_renabap")["fam_expuestas"]
    .sum()
    .reset_index()
)

# Merge with geometry
barrio_exposure_map = renabap_pba_intersect.merge(
    barrio_exposure, on="id_renabap", how="left"
)

# Plot choropleth
barrio_exposure_map.plot(
    column="fam_expuestas", 
    ax=ax4, 
    legend=True, 
    legend_kwds={'label': 'Familias expuestas'},
    cmap='Reds',
    zorder=5
)

plt.tight_layout()
plt.show()

## Referencias